<a href="https://colab.research.google.com/github/GVSU-CIS635/term-project-proposal-plato/blob/main/CIS635_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Thematic and Argument Structure Analysis of Plato’s Republic Using Data Mining Techniques
Tanishq Daniel, Trevor Ouma, Nate Miller

SyntaxError: invalid syntax (<ipython-input-1-16e56528afc1>, line 1)